In [ ]:
!nvidia-smi

In [ ]:
%%writefile dif_completo_metricas_cuda_final_corregido.cu

#include <iostream>
#include <fstream>
#include <cmath>
#include <chrono>
#include <algorithm>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <curand_kernel.h>
#include <thrust/device_ptr.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>
#include <iomanip>
#include <cstdlib>
#include <string>
#include <vector>
#include <filesystem>

using namespace std;
using namespace std::chrono;
namespace fs = std::filesystem;

// Configuración de la simulación
int N = 8192;
const int steps = 100000;
const int output_interval = 100;
const int save_interval = 1000;
const double dx = 1.0;
const double dx_sq = dx * dx;

// Parámetros en memoria constante
__constant__ double c_dt = 0.1;
__constant__ double c_Du = 0.20;
__constant__ double c_Dv = 0.10;
__constant__ double c_F = 0.026;
__constant__ double c_k = 0.053;
__constant__ double c_dx_sq = dx_sq;

// Configuración óptima para GPU
const int BLOCK_SIZE_X = 32;
const int BLOCK_SIZE_Y = 16;
const int TILE_PADDING = 1;
const int HIST_BINS = 64;
const int NUM_STREAMS = 4;

struct SimulationTimers {
    double initialization = 0.0;
    double simulation = 0.0;
    double transfer = 0.0;
    double saving = 0.0;
    double entropy = 0.0;
    double gradient = 0.0;
    double metrics = 0.0;
    double total = 0.0;
};

void run_simulation(int geometry_type, int num_sources, SimulationTimers& timers);
void print_geometry_options();
void calculate_max_grid_size(int& N, size_t available_memory);
size_t calculate_required_memory(int N);

#define CHECK_CUDA_ERROR(val) check((val), #val, __FILE__, __LINE__)
template <typename T>
void check(T err, const char* const func, const char* const file, const int line) {
    if (err != cudaSuccess) {
        cerr << "CUDA Error at: " << file << ":" << line << endl;
        cerr << cudaGetErrorString(err) << " " << func << endl;
        exit(1);
    }
}

__global__ void initialize_BZ_kernel(double* u, double* v, int N, int geometry_type, int num_sources);
__global__ void bz_simulation_step(double* u, double* v, double* u_next, double* v_next, int N);
__global__ void calculate_histogram(const double* data, int* hist, int N, double min_val, double bin_size);
__global__ void calculate_gradients(const double* data, double* grad_x, double* grad_y, int N);
__global__ void prepare_save_data(const double* v, float* v_out, int N);
__global__ void reduce_histogram(int* hist, int size);
__global__ void calculate_pattern_complexity(const double* v, double* complexity, int N);

int main() {
    cout << "=== Simulación de Reacción-Difusión Belousov-Zhabotinsky ===" << endl;

    cudaDeviceProp prop;
    CHECK_CUDA_ERROR(cudaGetDeviceProperties(&prop, 0));
    CHECK_CUDA_ERROR(cudaSetDevice(0));

    size_t free_mem, total_mem;
    CHECK_CUDA_ERROR(cudaMemGetInfo(&free_mem, &total_mem));

    cout << "\nInformación de la GPU:" << endl;
    cout << "Nombre: " << prop.name << endl;
    cout << "Memoria total: " << total_mem / (1024.0 * 1024.0) << " MB" << endl;
    cout << "Memoria disponible: " << free_mem / (1024.0 * 1024.0) << " MB" << endl;

    int max_N;
    calculate_max_grid_size(max_N, total_mem * 0.9);

    cout << "\nTamaño máximo teórico: " << max_N << " x " << max_N << endl;
    cout << "Ingrese el tamaño de malla deseado (máximo " << max_N << "): ";
    cin >> N;

    if (N <= 0 || N > max_N) {
        cerr << "Error: Tamaño no válido. Usando máximo." << endl;
        N = max_N;
    }

    size_t req_mem = calculate_required_memory(N);
    cout << "Tamaño seleccionado: " << N << " x " << N << endl;
    cout << "Memoria requerida: " << req_mem / (1024.0 * 1024.0) << " MB" << endl;

    print_geometry_options();

    int geometry_type, num_sources;
    cout << "Seleccione el tipo de geometría (1-5): ";
    cin >> geometry_type;

    if (geometry_type < 1 || geometry_type > 5) {
        cerr << "Error: Opción no válida." << endl;
        return 1;
    }

    if (geometry_type == 1) {
        cout << "Número de focos: ";
        cin >> num_sources;
    } else {
        num_sources = 1;
    }

    SimulationTimers timers;
    run_simulation(geometry_type, num_sources, timers);

    cout << "\n=== Resultados ===" << endl;
    cout << fixed << setprecision(4);
    cout << "Inicialización: " << timers.initialization << " s" << endl;
    cout << "Simulación: " << timers.simulation << " s" << endl;
    cout << "Transferencia: " << timers.transfer << " s" << endl;
    cout << "Guardado: " << timers.saving << " s" << endl;
    cout << "Entropía: " << timers.entropy << " s" << endl;
    cout << "Gradiente: " << timers.gradient << " s" << endl;
    cout << "Métricas: " << timers.metrics << " s" << endl;
    cout << "Total: " << timers.total << " s" << endl;

    return 0;
}

size_t calculate_required_memory(int N) {
    return 4 * N * N * sizeof(double) +
           2 * N * N * sizeof(float) +
           2 * N * N * sizeof(double) +
           N * N * sizeof(double) +
           HIST_BINS * sizeof(int) +
           NUM_STREAMS * sizeof(cudaStream_t);
}

void calculate_max_grid_size(int& N, size_t available_memory) {
    size_t per_element = 4 * sizeof(double) +
                         sizeof(float) +
                         2 * sizeof(double) +
                         sizeof(double);

    size_t additional_memory = HIST_BINS * sizeof(int) +
                               NUM_STREAMS * 2 * sizeof(cudaStream_t);

    size_t max_elements = (available_memory - additional_memory) / per_element;
    int max_N = static_cast<int>(sqrt(max_elements));
    max_N = (max_N / 256) * 256;

    const int MIN_N = 2048;
    const int MAX_N = 16384;

    N = max(MIN_N, min(max_N, MAX_N));
}

void run_simulation(int geometry_type, int num_sources, SimulationTimers& timers) {
    auto total_start = high_resolution_clock::now();

    string output_dir = "BZ_Geometry_" + to_string(geometry_type);
    try {
        fs::create_directories(output_dir);
    } catch (...) {
        cerr << "Error al crear directorio" << endl;
        return;
    }

    cudaStream_t streams[NUM_STREAMS];
    for (int i = 0; i < NUM_STREAMS; ++i) {
        CHECK_CUDA_ERROR(cudaStreamCreate(&streams[i]));
    }

    double *d_u, *d_v, *d_u_next, *d_v_next;
    size_t grid_size = N * N * sizeof(double);

    CHECK_CUDA_ERROR(cudaMalloc(&d_u, grid_size));
    CHECK_CUDA_ERROR(cudaMalloc(&d_v, grid_size));
    CHECK_CUDA_ERROR(cudaMalloc(&d_u_next, grid_size));
    CHECK_CUDA_ERROR(cudaMalloc(&d_v_next, grid_size));

    CHECK_CUDA_ERROR(cudaMemset(d_u, 0, grid_size));
    CHECK_CUDA_ERROR(cudaMemset(d_v, 0, grid_size));
    CHECK_CUDA_ERROR(cudaMemset(d_u_next, 0, grid_size));
    CHECK_CUDA_ERROR(cudaMemset(d_v_next, 0, grid_size));

    float *d_v_out;
    CHECK_CUDA_ERROR(cudaMalloc(&d_v_out, N * N * sizeof(float)));

    double *d_grad_x, *d_grad_y, *d_complexity;
    CHECK_CUDA_ERROR(cudaMalloc(&d_grad_x, N * N * sizeof(double)));
    CHECK_CUDA_ERROR(cudaMalloc(&d_grad_y, N * N * sizeof(double)));
    CHECK_CUDA_ERROR(cudaMalloc(&d_complexity, N * N * sizeof(double)));

    int *d_hist;
    CHECK_CUDA_ERROR(cudaMalloc(&d_hist, HIST_BINS * sizeof(int)));

    dim3 blockDim(BLOCK_SIZE_X, BLOCK_SIZE_Y);
    dim3 gridDim((N + BLOCK_SIZE_X - 1) / BLOCK_SIZE_X,
                 (N + BLOCK_SIZE_Y - 1) / BLOCK_SIZE_Y);

    auto init_start = high_resolution_clock::now();
    initialize_BZ_kernel<<<gridDim, blockDim>>>(d_u, d_v, N, geometry_type, num_sources);
    CHECK_CUDA_ERROR(cudaDeviceSynchronize());
    timers.initialization = duration_cast<duration<double>>(high_resolution_clock::now() - init_start).count();

    ofstream metrics(output_dir + "/metrics.csv");
    metrics << "Paso,Entropia,GradientePromedio,Complejidad\n";

    auto sim_start = high_resolution_clock::now();
    for (int n = 0; n <= steps; ++n) {
        int stream_id = n % NUM_STREAMS;
        bz_simulation_step<<<gridDim, blockDim, 0, streams[stream_id]>>>(d_u, d_v, d_u_next, d_v_next, N);

        swap(d_u, d_u_next);
        swap(d_v, d_v_next);

        if (n % output_interval == 0) {
            auto transfer_start = high_resolution_clock::now();
            CHECK_CUDA_ERROR(cudaDeviceSynchronize());

            CHECK_CUDA_ERROR(cudaMemsetAsync(d_hist, 0, HIST_BINS * sizeof(int), streams[0]));
            calculate_histogram<<<gridDim, blockDim, 0, streams[0]>>>(d_v, d_hist, N, 0.0, 1.0/HIST_BINS);
            reduce_histogram<<<1, HIST_BINS, 0, streams[0]>>>(d_hist, HIST_BINS);

            calculate_gradients<<<gridDim, blockDim, 0, streams[1]>>>(d_v, d_grad_x, d_grad_y, N);
            calculate_pattern_complexity<<<gridDim, blockDim, 0, streams[2]>>>(d_v, d_complexity, N);

            timers.transfer += duration_cast<duration<double>>(high_resolution_clock::now() - transfer_start).count();

            auto entropy_start = high_resolution_clock::now();
            int hist[HIST_BINS] = {0};
            CHECK_CUDA_ERROR(cudaMemcpyAsync(hist, d_hist, HIST_BINS * sizeof(int), cudaMemcpyDeviceToHost, streams[0]));

            double entropy = 0.0;
            const double total = (N-2)*(N-2); // Excluir bordes
            const double log_bins = log(HIST_BINS);
            for (int i = 0; i < HIST_BINS; ++i) {
                if (hist[i] > 0) {
                    double p = hist[i] / total;
                    entropy -= p * log(p);
                }
            }
            entropy /= log_bins;
            timers.entropy += duration_cast<duration<double>>(high_resolution_clock::now() - entropy_start).count();

            auto gradient_start = high_resolution_clock::now();
            thrust::device_ptr<double> grad_x_ptr(d_grad_x);
            thrust::device_ptr<double> grad_y_ptr(d_grad_y);
            thrust::device_ptr<double> complexity_ptr(d_complexity);

            double sum_grad_x = thrust::reduce(thrust::cuda::par.on(streams[1]), grad_x_ptr, grad_x_ptr + N * N);
            double sum_grad_y = thrust::reduce(thrust::cuda::par.on(streams[1]), grad_y_ptr, grad_y_ptr + N * N);
            double avg_grad = (sum_grad_x + sum_grad_y) / (2 * (N-2)*(N-2));

            double total_complexity = thrust::reduce(thrust::cuda::par.on(streams[2]), complexity_ptr, complexity_ptr + N * N);
            double avg_complexity = total_complexity / ((N-2)*(N-2));

            timers.gradient += duration_cast<duration<double>>(high_resolution_clock::now() - gradient_start).count();

            auto metrics_start = high_resolution_clock::now();
            metrics << n << "," << fixed << setprecision(6)
                   << entropy << "," << avg_grad << "," << avg_complexity << "\n";

            if (n % save_interval == 0) {
                auto save_start = high_resolution_clock::now();
                prepare_save_data<<<gridDim, blockDim, 0, streams[3]>>>(d_v, d_v_out, N);

                vector<float> v_out(N * N);
                CHECK_CUDA_ERROR(cudaMemcpyAsync(v_out.data(), d_v_out, N * N * sizeof(float), cudaMemcpyDeviceToHost, streams[3]));

                ofstream out(output_dir + "/bz_" + to_string(n) + ".bin", ios::binary);
                out.write(reinterpret_cast<char*>(v_out.data()), N * N * sizeof(float));
                timers.saving += duration_cast<duration<double>>(high_resolution_clock::now() - save_start).count();
            }

            timers.metrics += duration_cast<duration<double>>(high_resolution_clock::now() - metrics_start).count();

            cout << "\rPaso: " << n << "/" << steps
                 << " | Entropía: " << fixed << setprecision(4) << entropy
                 << " | ∇: " << avg_grad
                 << " | C: " << avg_complexity << flush;
        }
    }
    timers.simulation = duration_cast<duration<double>>(high_resolution_clock::now() - sim_start).count();

    for (int i = 0; i < NUM_STREAMS; ++i) {
        CHECK_CUDA_ERROR(cudaStreamDestroy(streams[i]));
    }

    cudaFree(d_u);
    cudaFree(d_v);
    cudaFree(d_u_next);
    cudaFree(d_v_next);
    cudaFree(d_v_out);
    cudaFree(d_hist);
    cudaFree(d_grad_x);
    cudaFree(d_grad_y);
    cudaFree(d_complexity);

    metrics.close();

    timers.total = duration_cast<duration<double>>(high_resolution_clock::now() - total_start).count();
}

__global__ void initialize_BZ_kernel(double* u, double* v, int N, int geometry_type, int num_sources) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i >= N || j >= N) return;

    // Forzar cero en los bordes
    if (i == 0 || i == N-1 || j == 0 || j == N-1) {
        u[i * N + j] = 0.0;
        v[i * N + j] = 0.0;
        return;
    }

    const double radius = 8.0;
    const double radius_sq = radius * radius;
    const double center = N / 2.0;
    const double hex_size = N / 5.0;
    const double hex_const = hex_size * 0.866;

    bool in_geometry = false;

    switch(geometry_type) {
        case 1: {
            const double angle_step = 2.0 * M_PI / num_sources;
            const double dist = N / 3.5;
            for (int s = 0; s < num_sources; s++) {
                double angle = angle_step * s;
                double cx = center + dist * cos(angle);
                double cy = center + dist * sin(angle);
                double dx = i - cx;
                double dy = j - cy;
                if (dx*dx + dy*dy < radius_sq) {
                    in_geometry = true;
                    break;
                }
            }
            break;
        }
        case 2: {
            int j_center = N / 2;
            if (j >= j_center - 3 && j <= j_center + 3) in_geometry = true;
            break;
        }
        case 3: {
            int size = N / 4;
            int i_start = N/2 - size;
            int i_end = N/2 + size;
            int j_start = i_start;
            int j_end = i_end;
            if (i >= i_start && i <= i_end && j >= j_start && j <= j_end) in_geometry = true;
            break;
        }
        case 4: {
            double dx_val = abs(i - center);
            double dy_val = abs(j - center);
            if (dx_val <= hex_size && dy_val <= hex_const &&
                (0.5*hex_size + 0.866*dy_val) <= hex_size) in_geometry = true;
            break;
        }
        case 5: {
            int center_start = N/2 - 2;
            int center_end = N/2 + 2;
            if ((j >= center_start && j <= center_end) ||
                (i >= center_start && i <= center_end)) in_geometry = true;
            break;
        }
    }

    if (in_geometry) {
        u[i * N + j] = 0.2;
        v[i * N + j] = 0.9;
    } else {
        curandState state;
        curand_init(clock64(), i * N + j, 0, &state);
        v[i * N + j] = 0.001 * curand_uniform(&state);
        u[i * N + j] = 0.8 + 0.05 * curand_uniform(&state);
    }
}

__global__ void bz_simulation_step(double* u, double* v, double* u_next, double* v_next, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    // Mantener bordes en cero
    if (i == 0 || i == N-1 || j == 0 || j == N-1) {
        u_next[i * N + j] = 0.0;
        v_next[i * N + j] = 0.0;
        return;
    }
    if (i >= N || j >= N) return;

    __shared__ double s_u[BLOCK_SIZE_Y + 2 * TILE_PADDING][BLOCK_SIZE_X + 2 * TILE_PADDING];
    __shared__ double s_v[BLOCK_SIZE_Y + 2 * TILE_PADDING][BLOCK_SIZE_X + 2 * TILE_PADDING];

    int tx = threadIdx.x + TILE_PADDING;
    int ty = threadIdx.y + TILE_PADDING;

    s_u[ty][tx] = u[i * N + j];
    s_v[ty][tx] = v[i * N + j];

    if (threadIdx.x < TILE_PADDING) {
        int left = (blockIdx.x * blockDim.x - TILE_PADDING + threadIdx.x + N) % N;
        s_u[ty][threadIdx.x] = u[left * N + j];
        s_v[ty][threadIdx.x] = v[left * N + j];

        int right = (blockIdx.x * blockDim.x + blockDim.x + threadIdx.x) % N;
        s_u[ty][tx + blockDim.x] = u[right * N + j];
        s_v[ty][tx + blockDim.x] = v[right * N + j];
    }

    if (threadIdx.y < TILE_PADDING) {
        int top = (blockIdx.y * blockDim.y - TILE_PADDING + threadIdx.y + N) % N;
        s_u[threadIdx.y][tx] = u[i * N + top];
        s_v[threadIdx.y][tx] = v[i * N + top];

        int bottom = (blockIdx.y * blockDim.y + blockDim.y + threadIdx.y) % N;
        s_u[ty + blockDim.y][tx] = u[i * N + bottom];
        s_v[ty + blockDim.y][tx] = v[i * N + bottom];
    }

    __syncthreads();

    double lap_u = (s_u[ty][tx+1] + s_u[ty][tx-1] +
                   s_u[ty+1][tx] + s_u[ty-1][tx] -
                   4.0 * s_u[ty][tx]) / c_dx_sq;

    double lap_v = (s_v[ty][tx+1] + s_v[ty][tx-1] +
                   s_v[ty+1][tx] + s_v[ty-1][tx] -
                   4.0 * s_v[ty][tx]) / c_dx_sq;

    double u_val = s_u[ty][tx];
    double v_val = s_v[ty][tx];
    double uvv = u_val * v_val * v_val;

    // Expresiones corregidas
    double new_u = u_val + c_dt * (c_Du * lap_u - uvv + c_F * (1.0 - u_val));
    u_next[i * N + j] = fmax(0.0, fmin(1.5, new_u));

    double new_v = v_val + c_dt * (c_Dv * lap_v + uvv - (c_F + c_k) * v_val);
    v_next[i * N + j] = fmax(0.0, fmin(1.0, new_v));
}

__global__ void calculate_histogram(const double* data, int* hist, int N, double min_val, double bin_size) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i == 0 || i == N-1 || j == 0 || j == N-1) return;
    if (i >= N || j >= N) return;

    double val = data[i * N + j];
    int bin = min(HIST_BINS - 1, (int)((val - min_val) / bin_size));
    atomicAdd(&hist[bin], 1);
}

__global__ void reduce_histogram(int* hist, int size) {
    int tid = threadIdx.x;
    if (tid >= size) return;

    for (int s = size / 2; s > 0; s >>= 1) {
        if (tid < s) {
            hist[tid] += hist[tid + s];
        }
        __syncthreads();
    }
}

__global__ void calculate_gradients(const double* data, double* grad_x, double* grad_y, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i == 0 || i == N-1 || j == 0 || j == N-1) {
        grad_x[i * N + j] = 0.0;
        grad_y[i * N + j] = 0.0;
        return;
    }
    if (i >= N || j >= N) return;

    int right = (i + 1) % N;
    int left = (i - 1 + N) % N;
    int top = (j - 1 + N) % N;
    int bottom = (j + 1) % N;

    grad_x[i * N + j] = fabs(data[right * N + j] - data[left * N + j]) / (2.0 * dx);
    grad_y[i * N + j] = fabs(data[i * N + bottom] - data[i * N + top]) / (2.0 * dx);
}

__global__ void calculate_pattern_complexity(const double* v, double* complexity, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i == 0 || i == N-1 || j == 0 || j == N-1) {
        complexity[i * N + j] = 0.0;
        return;
    }
    if (i >= N || j >= N) return;

    int right = (i + 1) % N;
    int left = (i - 1 + N) % N;
    int top = (j - 1 + N) % N;
    int bottom = (j + 1) % N;

    double lap = (v[right * N + j] + v[left * N + j] +
                 v[i * N + top] + v[i * N + bottom] -
                 4.0 * v[i * N + j]) / c_dx_sq;

    complexity[i * N + j] = fabs(lap);
}

__global__ void prepare_save_data(const double* v, float* v_out, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int j = blockIdx.y * blockDim.y + threadIdx.y;

    if (i >= N || j >= N) return;

    v_out[i * N + j] = static_cast<float>(v[i * N + j]);
}

void print_geometry_options() {
    const char* options =
        "================================\n"
        "    Geometrías disponibles:\n"
        "================================\n"
        "1. Focos circulares (especificar número)\n"
        "2. Línea horizontal central\n"
        "3. Cuadrado central\n"
        "4. Patrón hexagonal\n"
        "5. Cruz central\n"
        "================================\n";
    cout << options;
}

In [ ]:
!nvcc -std=c++17 -arch=sm_75 --expt-relaxed-constexpr -O3 dif_completo_metricas_cuda_final_corregido.cu -o simulacion_bz


In [ ]:
!./simulacion_bz

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FFMpegWriter
import glob
import os
from datetime import datetime
from google.colab import files
from IPython.display import HTML

# ===== CONFIGURACIÓN PRINCIPAL =====
plt.style.use('dark_background')
fig = plt.figure(figsize=(16, 9), facecolor='black')

# Definir posiciones absolutas para cada subplot
left_main = 0.05
bottom_main = 0.05
width_main = 0.65
height_main = 0.90

left_metrics = 0.72
bottom_entropy = 0.55
bottom_gradient = 0.05
width_metrics = 0.25
height_metrics = 0.35

ax_main = fig.add_axes([left_main, bottom_main, width_main, height_main])
ax_entropy = fig.add_axes([left_metrics, bottom_entropy, width_metrics, height_metrics])
ax_gradient = fig.add_axes([left_metrics, bottom_gradient, width_metrics, height_metrics])

# Paleta de colores
cmap = plt.get_cmap('inferno')
cmap.set_under('black')
cmap.set_over('white')

# ===== FUNCIONES AUXILIARES =====
def find_simulation_folder():
    """Encuentra la carpeta de simulación más reciente"""
    folders = sorted(glob.glob("BZ_Geometry_*"),
                   key=lambda x: os.path.getmtime(x),
                   reverse=True)
    return folders[0] if folders else None

def load_metrics(folder):
    """Carga los datos de métricas del archivo CSV"""
    metrics_file = f"{folder}/metrics.csv"
    if os.path.exists(metrics_file):
        try:
            data = np.genfromtxt(metrics_file, delimiter=',',
                               skip_header=1,
                               names=['step', 'entropy', 'gradient', 'complexity'])
            if np.max(data['entropy']) > 1:
                data['entropy'] = data['entropy'] / np.max(data['entropy'])
            return data
        except Exception as e:
            print(f"Error al cargar métricas: {str(e)}")
            return None
    return None

def load_bin_file(filename, N=None):
    """Carga un archivo binario y lo convierte en un array 2D de floats"""
    with open(filename, 'rb') as f:
        data = np.fromfile(f, dtype=np.float32)

    if N is None:
        # Calcular N asumiendo que es una matriz cuadrada perfecta
        N = int(np.sqrt(data.size))
        if N*N != data.size:
            raise ValueError(f"El tamaño de los datos ({data.size}) no corresponde a una matriz cuadrada perfecta")

    return data.reshape((N, N))

# ===== DETECCIÓN DE SIMULACIÓN =====
simulation_folder = find_simulation_folder()
if not simulation_folder:
    raise FileNotFoundError("No se encontraron carpetas de simulación BZ_Geometry_*")

print(f"\nProcesando simulación en: {simulation_folder}")

# Determinar tipo de geometría
geometry_type = simulation_folder.split('_')[-1] if '_' in simulation_folder else '5'
geometry_names = {
    '1': 'Focos Circulares',
    '2': 'Línea Horizontal',
    '3': 'Cuadrado Central',
    '4': 'Patrón Hexagonal',
    '5': 'Cruz Central'
}
geometry_name = geometry_names.get(geometry_type, f"Geometría {geometry_type}")

# ===== CARGA DE DATOS BINARIOS =====
bin_files = sorted(glob.glob(f"{simulation_folder}/bz_*.bin"),
                  key=lambda x: int(''.join(filter(str.isdigit, os.path.basename(x)))))

if not bin_files:
    raise FileNotFoundError(f"No se encontraron archivos .bin en {simulation_folder}")

# Cargar primer frame para determinar dimensiones
try:
    sample_data = load_bin_file(bin_files[0])
    N = sample_data.shape[0]
    print(f"Dimensiones de la simulación: {N}x{N}")
except Exception as e:
    raise ValueError(f"Error al cargar el primer archivo binario: {str(e)}")

# Ajustar tamaño de figura basado en el tamaño de la malla
aspect_ratio = N / N  # Asumiendo malla cuadrada
fig.set_size_inches(16, 9 * aspect_ratio)

# Pre-cálculo de los límites de color
print("\nCalculando rangos de color...")
sample_size = min(100, len(bin_files))
sample_indices = np.linspace(0, len(bin_files)-1, sample_size, dtype=int)
sample_data_list = []

for i in sample_indices:
    try:
        data = load_bin_file(bin_files[i], N)
        sample_data_list.append(data)
    except Exception as e:
        print(f"Error al cargar archivo {bin_files[i]}: {str(e)}")
        continue

if not sample_data_list:
    raise ValueError("No se pudieron cargar archivos binarios para determinar el rango de color")

all_data = np.concatenate([d.flatten() for d in sample_data_list])
global_min = np.percentile(all_data, 1)
global_max = np.percentile(all_data, 99)
print(f"Rango de color fijado: {global_min:.4f} - {global_max:.4f}")

# Carga de métricas
metrics_data = load_metrics(simulation_folder)
if metrics_data is not None:
    max_step = metrics_data['step'][-1] if len(metrics_data['step']) > 0 else len(bin_files)
    max_grad = np.max(metrics_data['gradient']) * 1.1 if len(metrics_data['gradient']) > 0 else 1.0
    max_entropy = np.max(metrics_data['entropy']) * 1.05 if len(metrics_data['entropy']) > 0 else 1.0
else:
    max_step = len(bin_files)
    max_grad = 1.0
    max_entropy = 1.0

# ===== CONFIGURACIÓN DE VIDEO =====
output_name = f"BZ_{geometry_name.replace(' ', '_')}_{N}x{N}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"

writer = FFMpegWriter(
    fps=20,
    codec='libx264',
    bitrate=8000,
    metadata={
        'title': f'BZ Simulation - {geometry_name}',
        'grid_size': f'{N}x{N}',
        'author': 'BZ Simulation Visualizer'
    },
    extra_args=[
        '-pix_fmt', 'yuv420p',
        '-profile:v', 'baseline',
        '-level', '3.0',
        '-crf', '18',
        '-preset', 'slow'
    ]
)

# ===== VISUALIZACIÓN INICIAL =====
ax_main.axis('off')
img = ax_main.imshow(sample_data, cmap=cmap,
                   vmin=global_min, vmax=global_max,
                   interpolation='bilinear',
                   origin='lower',
                   extent=[0, N, 0, N])

cbar = fig.colorbar(img, ax=ax_main, fraction=0.046, pad=0.04)
cbar.set_label('Concentración', rotation=270, labelpad=15, color='white')
cbar.ax.yaxis.set_tick_params(color='white')
plt.setp(cbar.ax.get_yticklabels(), color='white')

if metrics_data is not None:
    entropy_line, = ax_entropy.plot([], [], 'c-', lw=2, label='Entropía')
    gradient_line, = ax_gradient.plot([], [], 'm-', lw=2, label='Gradiente')

    ax_entropy.set_title('Entropía Normalizada', color='white', pad=10)
    ax_entropy.set_ylim(0, max_entropy)
    ax_entropy.set_xlim(0, max_step)
    ax_entropy.grid(alpha=0.3, color='gray')
    ax_entropy.tick_params(colors='white')
    ax_entropy.legend()

    ax_gradient.set_title('Gradiente Promedio', color='white', pad=10)
    ax_gradient.set_ylim(0, max_grad)
    ax_gradient.set_xlim(0, max_step)
    ax_gradient.grid(alpha=0.3, color='gray')
    ax_gradient.tick_params(colors='white')
    ax_gradient.legend()

info_text = ax_main.text(0.02, 0.95, '', transform=ax_main.transAxes,
                        color='white', fontsize=10,
                        bbox=dict(facecolor='black', alpha=0.7, edgecolor='white'))

# ===== FUNCIÓN DE ACTUALIZACIÓN =====
def update_frame(idx):
    try:
        # Cargar y preparar datos
        data = load_bin_file(bin_files[idx], N)
        step = int(''.join(filter(str.isdigit, os.path.basename(bin_files[idx]))))
        time = step * 0.1  # Asumiendo dt=0.1 como en la simulación

        # Actualizar visualización
        img.set_array(data)
        info_text.set_text(f"Geometría: {geometry_name}\nPaso: {step}\nTiempo: {time:.1f} s\nDimensión: {N}×{N}")

        # Actualizar métricas si están disponibles
        if metrics_data is not None:
            current_idx = min(idx, len(metrics_data['step'])-1)
            entropy_line.set_data(metrics_data['step'][:current_idx+1],
                                metrics_data['entropy'][:current_idx+1])
            gradient_line.set_data(metrics_data['step'][:current_idx+1],
                                 metrics_data['gradient'][:current_idx+1])

        return [img, info_text, entropy_line, gradient_line]

    except Exception as e:
        print(f"\nError en frame {idx} ({bin_files[idx]}): {str(e)}")
        raise e

# ===== GENERACIÓN DE VIDEO =====
print("\nIniciando renderizado...")
try:
    total_frames = len(bin_files)
    with writer.saving(fig, output_name, dpi=120):
        # Frame inicial
        update_frame(0)
        writer.grab_frame()

        # Resto de frames
        for idx in range(1, total_frames):
            try:
                update_frame(idx)
                writer.grab_frame()

                # Mostrar progreso
                if (idx+1) % 50 == 0 or idx == total_frames-1:
                    progress = 100*(idx+1)/total_frames
                    print(f"\rProgreso: {idx+1}/{total_frames} ({progress:.1f}%)", end='', flush=True)

            except Exception as e:
                print(f"\nError procesando frame {idx}: {str(e)}")
                continue

    print(f"\n\nVideo generado exitosamente: {output_name}")
    print(f"Tamaño de la malla: {N}x{N}")
    print(f"Total de frames procesados: {total_frames}")
    print(f"Rango de concentración: [{global_min:.4f}, {global_max:.4f}]")

    # Descargar automáticamente en Colab
    if 'google.colab' in str(get_ipython()):
        files.download(output_name)
    else:
        print(f"Archivo guardado en: {os.path.abspath(output_name)}")

except Exception as e:
    print(f"\nError crítico al generar video: {str(e)}")

    # Mostrar animación en el notebook como fallback
    from matplotlib.animation import FuncAnimation
    print("Intentando mostrar vista previa en el notebook...")

    # Limitar a 100 frames para la vista previa
    preview_frames = min(100, len(bin_files))
    anim = FuncAnimation(fig, update_frame, frames=preview_frames,
                        interval=50, blit=True)
    plt.close()

    # Mostrar en el notebook
    HTML(anim.to_html5_video())